In [1]:
#Reading Dataset

import pandas as pd
import os

from tqdm import tqdm

dataset_folder = os.path.join(os.getcwd(), 'dataset')
tqdm.pandas(desc='Progress Bar')

def df_from_csv(filename):
    f = os.path.join(dataset_folder, filename)
    return pd.read_csv(f, delimiter='\t|\n|,', engine='python')

# def calculate_tx(row):
#     if not pd.isna(row['risk_rank']):
#         temp_frame = xdp[(xdp['tx_date'] <= row['date']) & (xdp['cust_id'] == row['cust_id'])]
#         total_tx = (temp_frame['tx_amt'] * temp_frame['exchg_rate'])
#         row['total_tx'] = total_tx.sum()
#         row['avg_tx'] = total_tx.mean()
#         row['count_tx'] = total_tx.count()
#         row['std_tx'] = total_tx.std()
#         row['tx_per_day'] = row['count_tx']/row['date']+1
#     return row

def calculate_tx(row):
    if not pd.isna(row['risk_rank']):
        temp_frame = xdp[(xdp['tx_date'] <= row['date']) & (xdp['cust_id'] == row['cust_id'])]

        cr_type1 = temp_frame[(temp_frame['debit_credit'] == 'CR') & (temp_frame['tx_type'] == 1)]
        total_tx = (cr_type1['tx_amt'] * cr_type1['exchg_rate'])
        row['cr_1_total_tx'] = total_tx.sum()
        row['cr_1_avg_tx'] = total_tx.mean()
        row['cr_1_count_tx'] = total_tx.count()
        row['cr_1_std_tx'] = total_tx.std()
        row['cr_1_tx_per_day'] = row['cr_1_count_tx']/(row['date']+1)

        cr_type2 = temp_frame[(temp_frame['debit_credit'] == 'CR') & (temp_frame['tx_type'] == 2)]
        total_tx = (cr_type2['tx_amt'] * cr_type2['exchg_rate'])
        row['cr_2_total_tx'] = total_tx.sum()
        row['cr_2_avg_tx'] = total_tx.mean()
        row['cr_2_count_tx'] = total_tx.count()
        row['cr_2_std_tx'] = total_tx.std()
        row['cr_2_tx_per_day'] = row['cr_2_count_tx']/(row['date']+1)

        cr_type3 = temp_frame[(temp_frame['debit_credit'] == 'CR') & (temp_frame['tx_type'] == 3)]
        total_tx = (cr_type3['tx_amt'] * cr_type3['exchg_rate'])
        row['cr_3_total_tx'] = total_tx.sum()
        row['cr_3_avg_tx'] = total_tx.mean()
        row['cr_3_count_tx'] = total_tx.count()
        row['cr_3_std_tx'] = total_tx.std()
        row['cr_3_tx_per_day'] = row['cr_3_count_tx']/(row['date']+1)

        db_type1 = temp_frame[(temp_frame['debit_credit'] == 'DB') & (temp_frame['tx_type'] == 1)]
        total_tx = (db_type1['tx_amt'] * db_type1['exchg_rate'])
        row['db_1_total_tx'] = total_tx.sum()
        row['db_1_avg_tx'] = total_tx.mean()
        row['db_1_count_tx'] = total_tx.count()
        row['db_1_std_tx'] = total_tx.std()
        row['db_1_tx_per_day'] = row['db_1_count_tx']/(row['date']+1)

        db_type2 = temp_frame[(temp_frame['debit_credit'] == 'DB') & (temp_frame['tx_type'] == 2)]
        total_tx = (db_type2['tx_amt'] * db_type1['exchg_rate'])
        row['db_2_total_tx'] = total_tx.sum()
        row['db_2_avg_tx'] = total_tx.mean()
        row['db_2_count_tx'] = total_tx.count()
        row['db_2_std_tx'] = total_tx.std()
        row['db_2_tx_per_day'] = row['db_2_count_tx']/(row['date']+1)

        db_type3 = temp_frame[(temp_frame['debit_credit'] == 'DB') & (temp_frame['tx_type'] == 3)]
        total_tx = (db_type3['tx_amt'] * db_type3['exchg_rate'])
        row['db_3_total_tx'] = total_tx.sum()
        row['db_3_avg_tx'] = total_tx.mean()
        row['db_3_count_tx'] = total_tx.count()
        row['db_3_std_tx'] = total_tx.std()
        row['db_3_tx_per_day'] = row['db_3_count_tx']/(row['date']+1)
    return row

In [2]:
y_train = df_from_csv('ori_trainy.csv')
x_train = df_from_csv('ori_trainx.csv')
x_test = df_from_csv('submit_format.csv')
alert_cust = df_from_csv('ori_custinfo.csv')
alert_date = df_from_csv('ori_alert_date.csv')
xdp = df_from_csv('ori_xdp.csv')

In [3]:
xdp_final = x_train.merge(y_train, on='alert_key', how='inner')
xdp_final = xdp_final.merge(alert_cust, on='alert_key', how='inner')
xdp_final.isnull().any()

alert_key          False
date               False
sar_flag           False
cust_id            False
risk_rank          False
occupation_code     True
total_asset        False
AGE                False
dtype: bool

In [4]:
xdp_final['occupation_code'] = xdp_final['occupation_code'].fillna(20)
xdp_final['occupation_code'] = xdp_final['occupation_code'].astype(int)
xdp_final.isnull().any()

alert_key          False
date               False
sar_flag           False
cust_id            False
risk_rank          False
occupation_code    False
total_asset        False
AGE                False
dtype: bool

In [5]:
xdp_final.head()

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171189,0,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17,375576.0,4
1,171202,0,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12,2717416.0,2
2,171599,0,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12,326517.0,4
3,171737,0,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14,1014759.0,4
4,171142,0,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12,241719.0,3


In [6]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
xdp_final = pd.get_dummies(xdp_final, columns=['occupation_code'])
xdp_final

,alert_key,date,sar_flag,cust_id,risk_rank,total_asset,AGE,occupation_code_0,occupation_code_1,occupation_code_2,...,occupation_code_11,occupation_code_12,occupation_code_13,occupation_code_14,occupation_code_15,occupation_code_16,occupation_code_17,occupation_code_18,occupation_code_19,occupation_code_20
0,171189,0,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,375576.0,4,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,171202,0,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,2717416.0,2,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,171599,0,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,326517.0,4,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,171737,0,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,1014759.0,4,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,171142,0,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,241719.0,3,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,352132,364,0,c49b33d1fde790ec03584672903f296e486615adfdc989...,1,3218731.0,3,0,0,0,...,0,0,0,0,0,0,0,0,1,0
23902,352125,364,0,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,928963.0,3,0,0,0,...,0,0,0,0,0,0,0,0,1,0
23903,352080,364,0,1a93961c8fa830a1f32f5316b44f4964d65397f0311b11...,2,69080.0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23904,352075,364,0,9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33...,3,262604.0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
xdp_final = xdp_final.progress_apply(calculate_tx, axis=1)

Progress Bar: 100%|██████████| 23906/23906 [31:42<00:00, 12.57it/s]


In [8]:
xdp_final.to_pickle('xdp_x_train.pkl')

In [9]:
x_test = x_test.drop(['probability'], axis=1)
x_test = x_test.merge(alert_date, on='alert_key', how='left')
x_test = x_test.merge(alert_cust, on='alert_key', how='left')

x_test['occupation_code'] = x_test['occupation_code'].fillna(20)
x_test['occupation_code'] = x_test['occupation_code'].astype(int)
x_test = pd.get_dummies(x_test, columns=['occupation_code'])

x_test = x_test.progress_apply(calculate_tx, axis=1)
x_test

Progress Bar: 100%|██████████| 3850/3850 [02:25<00:00, 26.38it/s]


,AGE,alert_key,cr_1_avg_tx,cr_1_count_tx,cr_1_std_tx,cr_1_total_tx,cr_1_tx_per_day,cr_2_avg_tx,cr_2_count_tx,cr_2_std_tx,...,occupation_code_19,occupation_code_2,occupation_code_20,occupation_code_3,occupation_code_4,occupation_code_5,occupation_code_7,occupation_code_9,risk_rank,total_asset
0,3.0,357307,18913.500,2.0,3550.383148,37827.0,0.005391,1.812479e+04,95.0,5.605890e+04,...,0,0,0,0,1,0,0,0,1.0,2588452.0
1,NaN,376329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,NaN,NaN
2,NaN,373644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,NaN,NaN
3,5.0,357668,NaN,0.0,NaN,0.0,0.000000,4.722903e+05,218.0,4.005022e+06,...,1,0,0,0,0,0,0,0,1.0,1964540.0
4,3.0,354443,NaN,0.0,NaN,0.0,0.000000,2.793842e+05,129.0,4.999087e+05,...,0,1,0,0,0,0,0,0,1.0,444392.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3845,2.0,364485,51294.000,1.0,NaN,51294.0,0.002538,2.439224e+04,164.0,4.166776e+04,...,0,0,0,0,0,0,0,0,1.0,644169.0
3846,2.0,363155,4.000,1.0,NaN,4.0,0.002545,1.211959e+05,216.0,2.161619e+05,...,0,0,0,0,0,0,0,0,1.0,114439.0
3847,NaN,368710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,NaN,NaN
3848,3.0,358067,7486.875,8.0,12068.747342,59895.0,0.020888,2.297454e+06,126.0,6.546095e+06,...,0,0,0,0,0,0,0,1,1.0,367478.0


In [10]:
x_test.to_pickle('xdp_x_test.pkl')